# YOLO v3

In [1]:
#import libraries
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from yolov3 import *
from util import *

In [2]:
#From YOLOv3 paper https://pjreddie.com/media/files/papers/YOLOv3.pdf, bounding box priors
#dervived using k-means clustering on COCO dataset
ANCHORS = [(10,13),(16,30),(33,23),(30,61),(62,45),(59,119),(116,90),(156,198),(373,326)]



In [3]:
#Converting Official Yolo weights to Tensorflow format

"""
inputs = tf.placeholder(tf.float32, [1, 416, 416, 3])

model = YOLOv3((416,416), 80, ANCHORS, 0.5, 0.5, 'channels_last')
detect_model = model.detect(inputs)

yolovars = tf.global_variables(scope='yolov3')
assign_list = convert_weights(yolovars, './Data/yolov3.weights')

saver = tf.train.Saver(tf.global_variables(scope='yolov3'))

with tf.Session() as sess:
    sess.run(assign_list)
    saver.save(sess, './Data/yolov3.ckpt')
    print('Model has been saved successfully.')
"""

"\ninputs = tf.placeholder(tf.float32, [1, 416, 416, 3])\n\nmodel = YOLOv3((416,416), 80, ANCHORS, 0.5, 0.5, 'channels_last')\ndetect_model = model.detect(inputs)\n\nyolovars = tf.global_variables(scope='yolov3')\nassign_list = convert_weights(yolovars, './Data/yolov3.weights')\n\nsaver = tf.train.Saver(tf.global_variables(scope='yolov3'))\n\nwith tf.Session() as sess:\n    sess.run(assign_list)\n    saver.save(sess, './Data/yolov3.ckpt')\n    print('Model has been saved successfully.')\n"